# Import packages

In [2]:
import argparse
import os

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PromptEncoderConfig,
    PromptTuningConfig,
    PromptTuningInit,
)
from evaluate import load
from torch.utils.data import DataLoader, Dataset
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from tqdm import tqdm

# Read files

In [3]:
import pandas as pd
#load train data

def get_file(path):
    cols = ['id', 'text', 'label', 'intensity']
    anger = pd.read_csv(path + 'anger.txt', header=None, sep='\t', names=cols, index_col=0)
    fear = pd.read_csv(path + 'fear.txt', header=None, sep='\t', names=cols, index_col=0)
    sad = pd.read_csv(path + 'sadness.txt', header=None, sep='\t', names=cols, index_col=0)
    joy = pd.read_csv(path + 'joy.txt', header=None, sep='\t', names=cols, index_col=0)
    # Combine the DataFrames
    combined_df = pd.concat([anger, fear, sad, joy])
    # Shuffle the combined DataFrame
    shuffled = combined_df.sample(frac=1).reset_index(drop=True)
    return shuffled

train=get_file("train/")
val=get_file("Dev/")
train

text    label  intensity
0     Accept the challenges so that you can feel the...      joy      0.440
1     When you wake up from a dream laughing at some...      joy      0.875
2     @MoneyJay901 hard to tell in your pic. My mist...  sadness      0.646
3     When the sadness leaves you broken in your bed...     fear      0.479
4     It's Thursday which means it's Grey's day #TGI...      joy      0.560
...                                                 ...      ...        ...
3608  Sometimes I watch shitty tv to reinforce never...      joy      0.104
3609  Hey folks sorry if anything offensive got post...    anger      0.479
3610  @NewYorker @donlemon Trump presidency-a dark a...  sadness      0.604
3611  Bunk ur class📚 at least once in your life time...      joy      0.646
3612  Just got back from seeing @GaryDelaney in Burs...      joy      0.920

[3613 rows x 3 columns]

# Prepare the dataset

In [4]:
class EmotionDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.dataframe = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        sentence = self.dataframe.iloc[idx]['text']
        label = self.dataframe.iloc[idx]['label']

        # Tokenize the sentence
        inputs = self.tokenizer(sentence, truncation=True, max_length=None, return_tensors="pt")
        inputs = {key: val.squeeze(0) for key, val in inputs.items()}

        # Convert label to a numeric format if necessary
        label_to_id = {'joy': 0, 'anger': 1, 'fear': 2, 'sadness': 3}
        label_id = label_to_id[label]

        inputs['labels'] = torch.tensor(label_id, dtype=torch.long)

        return inputs

def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")


# Define the prefix tuning configurations

In [5]:
batch_size = 16
model_name_or_path = "cardiffnlp/twitter-roberta-base-emotion"
peft_type = PeftType.PROMPT_TUNING
device = "cuda"
num_epochs = 4
#note: PromptTuningInit only has TEXT and RANDOM, note sure why LABEL is not included even though it is recommended in the paper. 
peft_config = PromptTuningConfig(task_type="SEQ_CLS", 
                                 num_virtual_tokens=20,
                                 prompt_tuning_init=PromptTuningInit.TEXT,
                                prompt_tuning_init_text="Classify the emotions of the Tweets:",
                                tokenizer_name_or_path=model_name_or_path) 
lr = 0.005





# Define the pre-trained LLM

In [6]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,padding_side='left')
label_to_id = {'joy': 0, 'anger': 1, 'fear': 2, 'sadness': 3}
# Model
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, num_labels=len(label_to_id))

train_dataset = EmotionDataset(train, tokenizer)
val_dataset = EmotionDataset(val, tokenizer)
# Use the collate_fn in your DataLoaders
train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, shuffle=True, collate_fn=collate_fn, batch_size=batch_size)

### Print the model and peft details

In [8]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

trainable params: 609,028 || all params: 125,257,736 || trainable%: 0.48621986908656883


PeftModelForSequenceClassification(
  (base_model): PeftModelForSequenceClassification(
    (base_model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): 

# Define the optimizer

In [9]:
optimizer = AdamW(params=model.parameters(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

# Fine-tune!

In [10]:
model.to(device)
f1_metric = load('f1', config_name='multiclass', average='weighted')
accuracy_metric = load('accuracy')
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(val_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        accuracy_metric.add_batch(predictions=predictions, references=batch["labels"])
        f1_metric.add_batch(predictions=predictions, references=batch["labels"])

    # Compute final metric values
    final_accuracy = accuracy_metric.compute()
    final_f1 = f1_metric.compute(average='weighted')
    print(f"Accuracy: {final_accuracy['accuracy']}")
    print(f"F1 Score: {final_f1['f1']}")

  0%|                                                                                          | 0/226 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 60.72it/s]


Accuracy: 0.7002881844380403
F1 Score: 0.6995557986222193


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 61.02it/s]


Accuracy: 0.729106628242075
F1 Score: 0.7290246146962563


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 61.45it/s]


Accuracy: 0.7838616714697406
F1 Score: 0.7827939320760686


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 61.63it/s]

Accuracy: 0.792507204610951
F1 Score: 0.7914415635372353
